# Model I/O

LangChain provides the building blocks to interface with Language Models.

- **Language Models** — Make calls to language models through common interfaces.
- **Prompts** — Templatize, dynamically select, and manage model inputs
- **Output parsers** — Extract structured information from model outputs


## Language Models

- Make calls to language models through common interfaces.
- LangChain supports two types of model interfaces.
- **LLMs** 
    - Raw text completion models
    - Take input  of type `str` and give output of  type `str`.
- **Chat Models** 
    - Also based on LLMs but more tuned for having conversations between "Human" and "AI"
    - Take input of type `list[ChatMessage]` and give output of type `ChatMessage`
- However, both interfaces implement two base methods that allow to work seamlessly with either approach, so don't worry too much about it!
    - `model.predict()` — takes a `str` and returns a `str`
    - `model.predict_messages()` — takes a `list[ChatMessage]` and returns a `list[ChatMessage]`
    - `model.__call__()` — is the default method, it will use `predict()` or `predict_messages()` based on whether it's a LLM or a Chat Model.
- API is completely agnostic to the actual LLM used in the background (e.g. Claude, GPT-4, ...)
  - No need to know every different model API!

In [1]:
from dotenv import load_dotenv
import os
from pprint import pprint

In [2]:
# 1. Copy .env.example file as .env: `cp .env.example .env`
# 2. Open .env file and set all the env variables
load_dotenv(".env")
OPENAI_KEY = os.getenv("OPENAI_KEY")
assert OPENAI_KEY, "Please set your OPENAI_KEY environment variable."

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# The Chat Model
chat_model = ChatOpenAI(
    openai_api_key=OPENAI_KEY,
    model_name="gpt-4",
    temperature=0,
    model_kwargs={"top_p":1},
)

In [4]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming.")
]

response = chat_model(messages)

print(type(response))
pprint(vars(response))

<class 'langchain.schema.messages.AIMessage'>
{'additional_kwargs': {},
 'content': "J'aime la programmation.",
 'example': False,
 'is_chunk': False,
 'type': 'ai'}


## Prompts

- A prompt is a set of instructions or input provided to guide the model's response.
- A prompt can also help a model understand the context and generate relevant output.
- E.g. can guide towards different tasks:
  - answering questions
  - completing sentences
  - engaging in a conversation
  - ...
- Different classes and functions to help construct and work with prompts.
  - **Prompt templates**: Parametrized model inputs
  - **Example selectors**: Dynamically select examples to include in prompts, based on 
- **Prompt templates** are pre-defined recipes for generating prompts
  - e.g. include instructions, few-shots examples, ... for a given task
  - allow to re-use prompts across different calls!
  - fully model agnostic

In [5]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [6]:
template_str = """
Given the following 'Context' and 'Question', provide your 'Answer' as a
substring of the 'Context' or '[Unanswerable]' if the 'Question' cannot be answered
from the given 'Context'.

Follow closely the instructions, the output should be nothing different from what is asked.

'Context'
```
{context}
```

'Question'
```
{question}
```
"""

In [7]:
my_prompt_template = ChatPromptTemplate.from_template(template_str)

print(type(my_prompt_template))
pprint(vars(my_prompt_template))

<class 'langchain.prompts.chat.ChatPromptTemplate'>
{'input_variables': ['context', 'question'],
 'messages': [HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nGiven the following 'Context' and 'Question', provide your 'Answer' as a\nsubstring of the 'Context' or '[Unanswerable]' if the 'Question' cannot be answered\nfrom the given 'Context'.\n\nFollow closely the instructions, the output should be nothing different from what is asked.\n\n'Context'\n```\n{context}\n```\n\n'Question'\n```\n{question}\n```\n"))],
 'output_parser': None,
 'partial_variables': {}}


In [8]:
my_context = """
Switzerland, is geographically divided among the Swiss Plateau, the Alps and the Jura;
most of the country's population of 9 million are concentrated on the plateau,
which hosts the largest cities and economic centres, including Zürich, Geneva and Basel.
"""

my_question = "What is the capital of Switzerland?"

my_messages = my_prompt_template.format_messages(
context=my_context,
question=my_question,
)

print(type(my_messages))
print(type(my_messages[0]))
print(my_messages[0].content)

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>

Given the following 'Context' and 'Question', provide your 'Answer' as a
substring of the 'Context' or '[Unanswerable]' if the 'Question' cannot be answered
from the given 'Context'.

Follow closely the instructions, the output should be nothing different from what is asked.

'Context'
```

Switzerland, is geographically divided among the Swiss Plateau, the Alps and the Jura;
most of the country's population of 9 million are concentrated on the plateau,
which hosts the largest cities and economic centres, including Zürich, Geneva and Basel.

```

'Question'
```
What is the capital of Switzerland?
```



In [9]:
my_response = chat_model(my_messages)

print(type(my_response))
print(my_response.content)

<class 'langchain.schema.messages.AIMessage'>
[Unanswerable]


In [10]:
# Let's now do something with few-shot prompts.
examples = [
    {"input": "This is a terrible idea!", "output": "A"}, # class A = negative
    {"input": "I think we should evaluate this carefully.", "output": "B"}, # class B = neutral
    {"input": "This is the best idea ever!", "output": "C"}, # class C = positive
    {"input": "Awesome, let's do that!", "output": "C"}, # class C = positive
    {"input": "How could you even think of that???!!", "output": "A"}, # class A = negative
    {"input": "I'm not sure about that.", "output": "B"}, # class B = neutral
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: This is a terrible idea!
AI: A
Human: I think we should evaluate this carefully.
AI: B
Human: This is the best idea ever!
AI: C
Human: Awesome, let's do that!
AI: C
Human: How could you even think of that???!!
AI: A
Human: I'm not sure about that.
AI: B


In [11]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You must perform text classification. Given a sentence, predict its class as A, B, or C."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt[0])

prompt=PromptTemplate(input_variables=[], template='You must perform text classification. Given a sentence, predict its class as A, B, or C.')


In [12]:
my_prompt = final_prompt.format_messages(input="You're so sweet, thanks for that <3")
print(my_prompt)

[SystemMessage(content='You must perform text classification. Given a sentence, predict its class as A, B, or C.'), HumanMessage(content='This is a terrible idea!'), AIMessage(content='A'), HumanMessage(content='I think we should evaluate this carefully.'), AIMessage(content='B'), HumanMessage(content='This is the best idea ever!'), AIMessage(content='C'), HumanMessage(content="Awesome, let's do that!"), AIMessage(content='C'), HumanMessage(content='How could you even think of that???!!'), AIMessage(content='A'), HumanMessage(content="I'm not sure about that."), AIMessage(content='B'), HumanMessage(content="You're so sweet, thanks for that <3")]


In [13]:
chat_model(my_prompt)

AIMessage(content='C')

In [14]:
# The same, but using chains -- notice how the prompt flows through!
chain = final_prompt | chat_model

print(chain.invoke({"input": "You're so sweet, thanks for that <3"}))
print(chain.invoke({"input": "I hate you, you're so mean!"}))
print(chain.invoke({"input": "Mhe, not sure how to feel about that."}))

content='C'
content='A'
content='B'


## Output parsers

- The output of a language model is just a `str`. Even if the output of the model is e.g. `"3.14159"`, it's still just a `str` not a `float`.
- But often we want to extract structured information, rather than pure text.
- Output parsers help extracting structured information.
- These parsers must implement two methods.
  - `get_format_instructions()` — Instructions on how the LLM output should be formatted.
  - `parse(text: str)` — Take a `str` (assumed to be the output of a LLM) and parses it into some structure.
- There's also a third optional method.
  - `parse_with_prompt(completion: str, prompt: PromptValue)` — Take a `str` (assumed to be the output of a LLM) and a prompt (assumed to be the prompt which produced that output) and parses it into some structure.
- Into which structure can we parse the output of a LLM? Anything!
  - JSON, XML, datetime, enum, ...
- What is really cool is that Output Parsers can also be used to help you generate the formatting instructions to add to the Prompt!

In [15]:
import json

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator

In [16]:
# We will use Pydantic parser: this parser allows to specify an arbitrary JSON schema and query LLMs for JSON outputs that conform to that schema.
class MyOutput(BaseModel):
    answer: str = Field(description="The substring of the Context that answers the Question, or '' if the Question cannot be answered from the Context.")
    is_answerable: bool = Field(description="Whether the Question can be answered from the Context.")

In [17]:
output_parser = PydanticOutputParser(pydantic_object=MyOutput)


In [18]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"answer": {"title": "Answer", "description": "The substring of the Context that answers the Question, or '' if the Question cannot be answered from the Context.", "type": "string"}, "is_answerable": {"title": "Is Answerable", "description": "Whether the Question can be answered from the Context.", "type": "boolean"}}, "required": ["answer", "is_answerable"]}
```


In [19]:
  template_str = """
  For the following 'Context' and 'Question', extract the following information:

  - answer: The substring of the Context that answers the Question, or '' if the Question cannot be answered from the Context.
  - is_answerable: Whether the Question can be answered from the Context.

  'Context'
  ```
  {context}
  ```

  'Question'
  ```
  {question}
  ```

  {format_instructions}
  """

In [20]:
format_instructions = output_parser.get_format_instructions()

context_1 = """
Switzerland, is geographically divided among the Swiss Plateau, the Alps and the Jura;
most of the country's population of 9 million are concentrated on the plateau,
which hosts the largest cities and economic centres, including Zürich, Geneva and Basel.
"""

context_2 = """
Bern or Berne is the de facto capital of Switzerland, referred to by the Swiss as their "federal 
city". With a population of about 144,000, Bern is the fifth-most populous city in Switzerland.
"""

question = "What is the capital of Switzerland?"



In [21]:
prompt_template = ChatPromptTemplate.from_template(template=template_str)

responses = []
for i, context in enumerate((context_1, context_2), start=1):
    messages = prompt_template.format_messages(
        context=context,
        question=question,
        format_instructions=format_instructions,
    )
    response = chat_model(messages)
    responses.append(response.content)

In [22]:
print(responses)

['{"answer": "", "is_answerable": false}', '{"answer": "Bern or Berne", "is_answerable": true}']


In [23]:
parsed_responses = [output_parser.parse(response) for response in responses]
print(parsed_responses)

[MyOutput(answer='', is_answerable=False), MyOutput(answer='Bern or Berne', is_answerable=True)]


In [24]:
json_responses = [r.json() for r in parsed_responses]
print(json_responses)

['{"answer": "", "is_answerable": false}', '{"answer": "Bern or Berne", "is_answerable": true}']
